In [1]:
import torch
from dataloader import *
from trainers import *
from models import *
from helpers import *

In [2]:
device = "cpu"
if torch.cuda.is_available():
    print('running cuda...')
    device = "cuda"
elif torch.backends.mps.is_available():
    print('running mps...')
    device = "mps"

running cuda...


In [5]:
img_size = 320
MODEL = f'{img_size}_faster_rcnn'
json_file = f'dataloader/data/roboflow_chess_data/train/_annotations.coco.json'
root_folder = 'dataloader/data/roboflow_chess_data/train/'
PRETRAINED = True

In [6]:
from torch.utils.data import DataLoader
piece_data = PieceDetectorDataset(root_folder, json_file, (img_size, img_size))
dataloader = DataLoader(piece_data, 16, shuffle=True, collate_fn=lambda b: tuple(zip(*b)))

Piece Detector Dataset initialized!


In [7]:
for idx, (imgs, targets) in enumerate(dataloader):
    imgs = list(image.to(device) for image in imgs)
    # converts to a list of objects {boxes, labels, areas}
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    print(len(imgs))

16
16
16
16
16
16
16
16
16
16
16
16
10


In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes

In [ ]:
img, target = piece_data[0]
img = (img * 255).to(torch.uint8)
bbox_img = draw_bounding_boxes(img, target['boxes'])
plt.imshow(bbox_img.permute(1, 2, 0))
plt.show()

In [ ]:
target['boxes']

In [17]:
import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size(), obj.is_cuda)
    except:
        pass

<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([3]) False
<class 'torch.Tensor'> torch.Size([3]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([1]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([25, 3]) False
<class 'torch.Tensor'> torch.Size([23, 3]) False
<class 'torch.Tensor'> torch.Size([25, 2]) False
<class 'torch.Tensor'> torch.Size([23, 2]) False
<class 'torch.Tensor'> torch.Size([]) False
<class 'torch.Tensor'> torch.Size([3, 320, 320]) True
<class 'torch.Tensor'> torch.Size([3, 320, 320]) True
<class 'torch.Tensor'> torch.Size([3, 320, 320]) True
<class 'torch.Tensor'> 

In [21]:
torch.cuda.memory_allocated(0)

9750144000

In [9]:
epochs = 10
batch_size = [16]
lr = [1e-4]
weight_decay = 0.0005
writer = SummaryWriter(f'models/checkpoints/piece_detector/{MODEL}/tensorboard0')

for b in batch_size:
    for l in lr:
        loss, lowest_loss = train_piece_detector(batch_size=b,  # hyperparameter search (batch_size)
                                                   epochs=epochs,
                                                   weights_load_path=f'models/checkpoints/piece_detector/{MODEL}/weight',
                                                   weights_save_folder=f'./models/checkpoints/piece_detector/{MODEL}',
                                                   weights_name='weight0',
                                                   json_file=json_file,
                                                   root_folder=root_folder,
                                                   img_size=(img_size, img_size),
                                                   learning_rate=l,  # hyperparameter search (learning_rate)
                                                   weight_decay=weight_decay,
                                                   from_pretrained=PRETRAINED,
                                                   mixed_precision_training=torch.cuda.is_available(),
                                                   writer=writer,  # optional summary writer added!
                                                   step=0,
                                                   device=device)

        # writer.add_hparams({'learning rate': l, 'batch size': b},
        #                    {'lowest loss': lowest_loss, 'last loss': loss, 'mAP': m_ap['map']})
writer.close()
print('Training Complete!')

loaded weights for piece detector!
weights will be saved at./models/checkpoints/piece_detector/320_faster_rcnn/weight0 for piece detector
Piece Detector Dataset initialized!


  0%|          | 0/10 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 264.00 MiB (GPU 0; 12.00 GiB total capacity; 9.69 GiB already allocated; 0 bytes free; 9.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF